## Large Scale Data Mining assignment 1
### Similar items

Francisco Marques 97639 - Mestrado em Ciência de Dados
franciscocmarques@ua.pt


**IMPORTANT:** please download signature matrix in .npy format from: https://github.com/FMarques8/LargeScaleDataMining/blob/aa6e20524646e6730dba4c1fbbfd9b0b3559d9b6/sig_mat.npy (about 60 MBs)

In [1]:
%pip install pyspark
%pip install sympy
%pip install xxhash 
%pip install tqdm 
%pip install numpy
%pip install scipy
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext(appName="Assignment1 - News Articles")
print(sc.version)

3.3.2


In [3]:
spark = SparkSession.builder.appName("Assignment1 - News Articles").getOrCreate()
spark

### 2.1 Tuning $b$ bands and $r$ rows 

Computed the number of bands and rows in order to obtain at least $90\%$ of pairs with $85\%$ similarity and less than $5\%$ of pairs with $60\%$ similarity. Although we obtained around $79\%$ similarity, we can consider this a good value thanks to the accuracy vs complexity trade off. Reaching a similarity threshold closer to $85\%$ would increase the computational load exponentially.

In [4]:
def tune_params(bands, rows, sim_threshold):

  sim = (1/bands)**(1/rows)
  print(f"We obtained a s_threshold of {sim}, should be close to {sim_threshold}, with {rows} rows and {bands} bands.")

  sim_probability = 0.85
  p1 = sim_probability ** rows
  print(f"Probability of C1 and C2 identical in a given band = s^r = {p1}")

  p2 = (1 - p1) ** bands
  print(f"Probability C1, C2 are not similar in all of the {bands} bands = (1-s^r)^b = {p2}")
  if p2 < .1:
      print("Condition 1 obtained\n")

  not_sim_probability = 0.6
  p3 = not_sim_probability ** rows
  print(f"Probability C1, C2 are identical in a given band = s^r = {p3}")

  p4 = 1-(1-p3) ** bands
  print(f"Probability C1, C2 identical in at least 1 of 20 bands: 1-(1-s^r)^b = {p4}")
  if p4 < 0.05:
      print("Condition 2 obtained")

In [5]:
tune_params(13, 11, 0.85)

We obtained a s_threshold of 0.7920132050096789, should be close to 0.85, with 11 rows and 13 bands.
Probability of C1 and C2 identical in a given band = s^r = 0.1673432436896142
Probability C1, C2 are not similar in all of the 13 bands = (1-s^r)^b = 0.09248219617246255
Condition 1 obtained

Probability C1, C2 are identical in a given band = s^r = 0.0036279705599999985
Probability C1, C2 identical in at least 1 of 20 bands: 1-(1-s^r)^b = 0.0461505019889491
Condition 2 obtained


### 2.2 Function that, given an article, finds every article at least $85\%$ similar.

For this, we need to implement the full Locally-Sensitive Hashing (LSH) algorithm, starting with the Shingling and hashing the resulting shingles.

Here we can see the structure of each JSON entry.

In [6]:
df = spark.read.json('data/covid_news_full.json.bz2')
df.printSchema()
df.show()

root
 |-- text: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- url: string (nullable = true)

+--------------------+-------------------+--------------------+
|                text|           tweet_id|                 url|
+--------------------+-------------------+--------------------+
|Os especialistas ...|1215246971079929857|https://www.dn.pt...|
|Os leitores são a...|1215233921425854465|https://publico.p...|
|Os leitores são a...|1223386542145724416|https://www.publi...|
|Segundo a Associa...|1223380605897003008|https://www.cmjor...|
|Os leitores são a...|1223378524830097408|https://www.publi...|
|Os leitores são a...|1223375795999191040|https://publico.p...|
|(Enviada diariame...|1223373056229494784|https://www.cmjor...|
|De acordo com dad...|1223371884273766402|https://www.cmjor...|
|                    |1223368728940818440|https://www.publi...|
|Os leitores são a...|1223368343710838784|https://publico.p...|
|(Enviada diariame...|1223355638501257218|https://w

#### Shingling

Before we can shingle the documents, we need to clean the entries. For this, every punctuation and stopword was removed. As this algorithm can take a long time, we make use of *tqdm*'s progress bar.

In [7]:
import numpy as np
from tqdm import tqdm # progress bar
from nltk.corpus import stopwords
from string import punctuation
from sympy import nextprime
import json

np.set_printoptions(threshold=np.inf, precision=0, suppress=True) # prevents scientific notation when printing

After cleaning the entries, we remove any entry smaller than the shingle size. Since we considered a somewhat large shingle size ($k=10$) in this notebook, entries smaller than this, were in the few cases that I looked into, always the same ("(Enviada diariamente)" appeared many times).

Then the shingling itself, is quite simple. We made each document's shingles into a set to remove any duplicate as duplicates wont aid in finding similar items.

In [8]:
shingle_size = 10

punctuation_table = str.maketrans(dict.fromkeys(punctuation, '')) # Table to remove punctuation from text
stop_words = set(stopwords.words('portuguese')) # Set with stop words

news_rdd = df.rdd.map(lambda item: (item['tweet_id'], item['text'])) # Removes url

shingle_rdd = (news_rdd.map(lambda item: (item[0], item[1].translate(punctuation_table).split()))
                   # Remove punctuation and stop words
                .map(lambda item: (item[0], [w for w in item[1] if w.lower() not in stop_words]))
                # Filter out articles with less than 'shingle_size' words
                .filter(lambda item: len(item[1]) >= shingle_size) 
                # Shingle the documents
                .map(lambda item: (item[0], set([tuple(item[1][i:i + shingle_size]) for i in range(len(item[1]) - shingle_size+1)])))
                ) 

# number of total distinct shingles to use in the hash functions
num_shingles = shingle_rdd.flatMap(lambda item: item[1]).distinct().count()

Here we define the hash function and the permutations, which are constant throughout the algorithm. We use *sympy*.nextprime(x) to find the smallest prime greater than x. The hash function is the same as in the class slides: $((a*h(x)+b)\mod{p})\mod{N}$ where $N$ is the number of shingles, and $p$ the next smallest prime number.

In [9]:
p = nextprime(num_shingles)

num_permutations = 13 * 11 # bands x rows obtained from 2.1. b*r=n; n is the number of permutations
a = np.random.randint(1, p, size = (num_permutations))
b = np.random.randint(0, p, size = (num_permutations))

a_bc = sc.broadcast(a)
b_bc = sc.broadcast(b)

def hash_function(x, i):
  return ((a_bc.value[i] * hash(x) + b_bc.value[i]) % p) % num_shingles

With this we hash every shingle in each document and turn that list to a set, in order to remove duplicates. 

We also print some variables used throughout the algorithm such as the total number of unique shingles and number of articles. The original size is around 59 thousand, so with the previous filter we removed alot of small entries which can help speed up the algorithm.

In [10]:
# Hash shingles and convert list to set to remove duplicates
hashed_shingles = shingle_rdd.map(lambda item: (item[0], set([hash(shingle) for shingle in item[1]])))
num_docs = hashed_shingles.count()

print(f'Number of unique shingles: {num_shingles}')
print(f'Number of documents: {num_docs}')

Number of unique shingles: 7254120
Number of documents: 54446


#### MinHashing

Now that we have our shingles, we can perform the minhashing and create our signature matrix. This matrix will have the shape ('num_permutations', 'num_articles'), i.e., (143, 54446).

For this, we use the permutation values and hash every shingle in each document, then we append the minimum value to the 'minhash_values' list, and repeat until it is done 'num_permutations' times.

In [11]:
def minhash(document):
    """Applies minhashing to document."""
    minhash_values = []
    
    # For each permutation, get hash for every shingle and keep lowest
    for i in range(num_permutations):
        permuted_hash = [hash_function(shingle, i) for shingle in document]
        minhash_values.append(min(permuted_hash))
    
    return minhash_values

Using the speed up method of splitting the hashing into blocks, we split it into blocks of 500 entries, or 109 blocks.

We also repartition the shingles RDD (hashed shingles), and cache it since it will be accesses 109 times.

Then, every time a block is completed, we append its transpose (a column), to 'signature_matrix_blocks', which after every block is completed, concatenates them, creating our signature matrix.

In [12]:
def signature_matrix(shingles, num_docs, block_size):
    """Create signature matrix using minhash."""
    signature_matrix = []
    signature_matrix_blocks = []
    
    # Repartition RDD to lower bound (2 * cores) to reduce overhead and speed up process
    shingles_with_ids = shingles.zipWithIndex().map(lambda x: (x[1], x[0])).repartition(32).cache()

    # Process the data in blocks
    for i in tqdm(range(0, num_docs, block_size), desc = 'Hashing blocks'):
        # Get the next block of data
        block = shingles_with_ids.filter(lambda x: x[0] >= i and x[0] < i + block_size).map(lambda doc: doc[1][1])
        
        # Compute the MinHash values for each document in the block
        minhash_block = block.map(lambda doc: minhash(doc))
        signature_matrix_blocks.append(np.array(minhash_block.collect()).T)

    signature_matrix = np.concatenate(signature_matrix_blocks, axis = 1)

    return signature_matrix

In [13]:
sig_mat = signature_matrix(hashed_shingles, num_docs, 500)

Hashing blocks: 100%|██████████| 109/109 [59:53<00:00, 32.97s/it] 


In [63]:
# Save signature matrix to a file
np.save('sig_mat.npy', sig_mat)

# To load it
# sig_mat = np.load('sig_mat.npy')

#### LSH

Finally, we can generate our candidate pairs with LSH, but first we still need to split our entries into buckets.

For this we split our signature matrix into $b$ (13) bands, and in each band hash each column to one hash value, this is, we group the rows of a column in a band into a tuple, and hash that tuple. Then iterating over the hashes that were created, we split them into buckets, where the key is the hash value, this way if they have the same hash value, it means that the documents in that bucket are potential similar. Lastly, we add these buckets to a list, and repeat for the remaining bands, keeping each band's buckets *away* from the remaining. 

After every entry is distributed in the buckets, we go over each band's buckets and create every possible pair for buckets with at least 2 elements.

In [64]:
from itertools import combinations

def lsh(signature_matrix, num_docs, num_bands):
    # Split signature matrix into bands
    bands = np.split(signature_matrix, num_bands)
    buckets = []
    
    # Hash buckets by bands
    for band in tqdm(bands, desc = 'Bucket hashing'):
        # Hash each column r rows at a time
        band_hashes = [hash_function(tuple(band[:, j]), 0) for j in range(num_docs)]
        bucket_dict = {}
        
        # Distribute hashes over buckets
        for j, b_hash in enumerate(band_hashes):
            if b_hash not in bucket_dict:
                bucket_dict[b_hash] = []
            bucket_dict[b_hash].append(j)
        buckets.append(bucket_dict)
    
    # Generate candidate pairs if they are in the same bucket
    candidate_pairs = set()
    for bucket in tqdm(buckets, desc = 'Candidate pairs generation'):
        for b_hash, docs in list(bucket.items()):
            if len(docs) < 2:
                continue
            for pair in combinations(docs, 2):
                candidate_pairs.add(pair)

    return list(candidate_pairs)

Now we call the function and print the total number of candidates, and we can see that we have many candidates, which we will confirm if they are false positives or not.

In [65]:
candidate_pairs = lsh(sig_mat, num_docs, 13)

print(f'Total candidate pairs found: {len(candidate_pairs)}')

Candidate pairs generation: 100%|██████████| 13/13 [00:04<00:00,  2.86it/s]


Total candidate pairs found: 229154


Here we create out *jaccard* similarity function, and the function to find similar items in the candidate pairs. Since this function can be considered brute force since we're making sure if they are similar or not, it will take much longer than the previous algorithms.

In [20]:
def jaccard(set1: set[int], set2: set[int]):
    """Jaccard similarity between set1 and set2."""
    return len(set1.intersection(set2)) / len(set1.union(set2))

def get_similar(id, candidate_pairs, threshold, idx_to_id, hashed_rdd):
    """Find similar articles to the input based on the Jaccard similarity."""
    # Unpack pairs 
    candidate_documents = set([x for pair in candidate_pairs for x in pair])
    candidates_rdd = hashed_rdd.filter(lambda item: item[1] in candidate_documents).collect() # Collect 'real' candidates
    
    document_dict = {key: x for x, key in candidates_rdd} # Dict of document data -> {index: (id, shingles)}
    similar_articles = []

    for pair in candidate_pairs:
        if id in pair:
            sim = jaccard(document_dict[pair[0]][1], document_dict[pair[1]][1])
            if sim >= threshold:
                suggestion_id = set(pair) - set((id,))
                similar_articles.append(idx_to_id[suggestion_id.pop()])
    return similar_articles

### 2.3 Performance Evaluation
Now we must test the performance of our implementation, by running for several random samples and obtain false positives and false negatives. With this first cell we can verify if the candidates really are similar and compute the number of false positives. The approach for the false negatives is a bit different since we need to create every pair with a given article, and then compute their similarity.

In [62]:
# Dictionary with articles index and id
idx_to_id = {key: id[0] for id, key in hashed_shingles.zipWithIndex().collect()}
cached_hashed_shingles = hashed_shingles.zipWithIndex().repartition(32).cache()

# Unpack random 200 pairs
random_candidates = set([candidate_pairs[idx][0] for idx in np.random.choice(len(candidate_pairs), 200)])

# Make sure there are 200 candidates
while len(random_candidates) < 200:
    random_candidates.add(candidate_pairs[np.random.randint(0, len(candidate_pairs), 1)[0]][0])

suggestions = {idx_to_id[id]: [] for id in random_candidates}

# Verify how many candidate pairs are similar, save to dictionary regardless of being similar or not
for candidate in tqdm(random_candidates):
    similar_articles = get_similar(candidate, candidate_pairs, 0.85, idx_to_id, cached_hashed_shingles)
    candidate_id = idx_to_id[candidate] # Convert the index to the article id
    suggestions[candidate_id].extend(similar_articles)

100%|██████████| 200/200 [54:17<00:00, 16.29s/it]


Save the dictionary to a JSON file.

In [64]:
with open('suggestions.json', 'w') as f:
    json.dump(suggestions, f)

In [12]:
# Load with
with open('suggestions.json', 'r') as f:
    suggestions = json.load(f)

False negatives, this takes a long time to compute as it iterates over the whole data set. This will take up a lot of memory since it will load the entire hashed shingles RDD to memory.

In [60]:
# Choose 200 random ids to test the function
random_ids = set([idx for idx in np.random.choice(len(candidate_pairs), 100)])

# Make sure there are 100 candidates
while len(random_candidates) < 100:
    random_candidates.add(np.random.randint(0, len(candidate_pairs), 1))

# If loading JSON, need to add way to reverse back to index
random_keys = suggestions

cached_hashed_shingles = hashed_shingles.zipWithIndex().repartition(32).cache()
idx_to_id = {key: id for id, key in cached_hashed_shingles.collect()} # Index to document id 

id_to_idx = {key[0]: id for id, key in idx_to_id.items()} # convert back to index

random_ids = {id_to_idx[id]: vals for id, vals in random_keys.items()}

# Dictionary with similarities
sim_dict = {id: {'similar': [], 'not similar': []} for id in random_ids} 

for id in tqdm(random_ids):
    possible_pairs = set([(id, idx) for idx in range(num_docs) if id != idx])

    for pair in possible_pairs:
        sim = jaccard(idx_to_id[pair[0]][1], idx_to_id[pair[1]][1])
        if sim >= 0.85:
            sim_dict[id]['similar'].append(sim)
        else:
            sim_dict[id]['not similar'].append(sim)

{29184: [], 14340: [], 31749: [], 3078: [], 13321: [], 11788: [], 33298: [], 29202: ['1241289993315893248'], 9239: [], 46105: [], 8731: [], 14878: [], 16416: [], 8742: [], 23085: [], 12849: [], 3122: [], 19507: [], 2046: [], 50232: [], 2112: [], 48708: [], 6212: [], 21574: [], 4167: [], 12868: [], 28742: [], 42569: [], 13388: [], 50258: [], 3154: [], 19028: [], 32341: [], 21081: [], 1116: [], 25701: [], 16487: [], 33896: [], 4199: ['1242185523147485187'], 4207: [], 5231: [], 4212: [], 9335: [], 43640: [], 42105: [], 7290: [], 53247: [], 50809: [], 19581: [], 5760: [], 21639: [], 38025: [], 2186: [], 9355: [], 36494: [], 21135: [], 156: [], 1181: [], 43166: [], 37025: [], 32933: [], 34981: [], 1191: [], 15013: [], 2725: [], 12458: [], 13997: [], 22704: [], 12464: [], 39096: [], 12990: [], 46270: [], 1216: [], 3777: [], 49346: [], 17598: [], 26313: [], 40137: [], 18644: [], 9434: [], 42205: [], 19677: [], 8415: [], 48865: [], 33507: [], 6376: [], 237: [], 40174: [], 1776: [], 35568: [], 

100%|██████████| 200/200 [06:15<00:00,  1.88s/it]


In [69]:
performance = {id: {'FP': 0, 'FN': 0} for id in random_ids.keys()}

for id in tqdm(random_ids):
    for pair in candidate_pairs:
        if id in pair:
            if pair[0] in sim_dict[id]['not similar'] or pair[1] in sim_dict[id]['not similar']:
                performance[id]['FP'] += 1
        performance[id]['FN'] = len(sim_dict[id]['similar']) - performance[id]['FP']


100%|██████████| 200/200 [04:30<00:00,  1.35s/it]


We can finally take a look at the performance, and we can see that there are many False Negatives, which means that the LSH method did not have the best performance, one of the ways we could get better results would be to get closer to the 85% similarity by changing the number of bands and rows, since they were tuned for about 79%.

In [72]:
for idx, metrics in performance.items():
    print(idx_to_id[idx][0], metrics)

1318288932296065024 {'FP': 0, 'FN': 0}
1248917588308000768 {'FP': 0, 'FN': 0}
1329709945248419841 {'FP': 0, 'FN': 0}
1241421869590491136 {'FP': 0, 'FN': 2}
1255795486415609856 {'FP': 0, 'FN': 1}
1245413319336435712 {'FP': 0, 'FN': 1}
1325318690691293184 {'FP': 0, 'FN': 1}
1318244017772036097 {'FP': 0, 'FN': 158}
1246349622861275136 {'FP': 0, 'FN': 1}
1388625937651179520 {'FP': 0, 'FN': 1}
1251780613151096834 {'FP': 0, 'FN': 0}
1265983033263435778 {'FP': 0, 'FN': 0}
1258364289766785024 {'FP': 0, 'FN': 0}
1251773860032151553 {'FP': 0, 'FN': 0}
1282019664185491458 {'FP': 0, 'FN': 2}
1247185742859730947 {'FP': 0, 'FN': 6}
1241390069774266369 {'FP': 0, 'FN': 0}
1275726780800925699 {'FP': 0, 'FN': 0}
1239335744092725249 {'FP': 0, 'FN': 1}
1422217217219964932 {'FP': 0, 'FN': 0}
1239262050809008132 {'FP': 0, 'FN': 1}
1410516950409781248 {'FP': 0, 'FN': 0}
1242818687633182720 {'FP': 0, 'FN': 0}
1268529078518366208 {'FP': 0, 'FN': 1}
1240393283047763974 {'FP': 0, 'FN': 202}
1250194740844400641 {